In [ ]:
!pip install aiogram

In [ ]:
!pip install nest-asyncio

In [ ]:
import nest_asyncio
nest_asyncio.apply()
__import__('IPython').embed()

Python 3.7.12 (default, Sep 10 2021, 00:21:48) 
Type "copyright", "credits" or "license" for more information.

IPython 5.5.0 -- An enhanced Interactive Python.
?         -> Introduction and overview of IPython's features.
%quickref -> Quick reference.
help      -> Python's own help system.
object?   -> Details about 'object', use 'object??' for extra details.

In [1]: exit()



For exit from IPython type exit()

In [ ]:
!pip install timm

In [ ]:
from typing import Callable, Dict, Mapping, Tuple, Optional, Union, Any;from functools import partial;import os
import matplotlib.pyplot as plt;import random
import logging; from aiogram import Bot, Dispatcher, executor, types; from os import mkdir;import json
import torch;import cv2;import numpy as np; import pandas as pd;import albumentations as albu;from torch import nn; from torch.nn.modules.pooling import AdaptiveAvgPool2d; from torch.nn.modules.dropout import Dropout;from torch.nn.modules.linear import Linear;from timm.models.efficientnet import efficientnet_b2;from timm.models.resnet import resnet18, resnet34, resnet50

In [ ]:
user_lang = {}
C2LP = './Web app/app/class2label.json'  # path to class2label.json
MW = './Web app/app/best.pth'  # path to best.pth
DCEN = './Web app/app/desc_en.json'
DCRU = './Web app/app/desc_ru.json'

In [ ]:
def set_global_seed(seed: int, is_cudnn_deterministic: bool = True) -> None:
    """Setting seed for reproducible results.

    :param seed: seed number (no matter which one)
    :param is_cudnn_deterministic: is the algorithm determined on сuda or not
    """
    random.seed(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = is_cudnn_deterministic
    torch.backends.cudnn.benchmark = False

In [ ]:
ENCODERS = {
    'resnet18': {
        'features': 512,
        'init_op': partial(resnet18, pretrained=True),
    },
    'resnet34': {
        'features': 512,
        'init_op': partial(resnet34, pretrained=True),
    },
    'resnet50': {
        'features': 2048,
        'init_op': partial(resnet50, pretrained=True),
    },
    'efficientnet_b0': {
        'features': 1408,
        'init_op': partial(efficientnet_b2, pretrained=True),
    },
}


class SignsClassifier(nn.Module):
    """
    A model for classifying signs.
    """

    def __init__(self, encoder_name: str, n_classes: int, dropout_rate: float = 0.2):
        """Initializing the class.

        :param encoder_name: name of the network encoder
        :param n_classes: number of output classes
        :param dropout_rate: dropout rate
        """
        super().__init__()
        self.encoder = ENCODERS[encoder_name]['init_op']()
        self.avg_pool = AdaptiveAvgPool2d((1, 1))
        self.dropout = Dropout(dropout_rate)
        self.fc = Linear(ENCODERS[encoder_name]['features'], n_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Getting the model prediction.

        :param x: input batch tensor
        :return: prediction
        """
        x = self.encoder.forward_features(x)
        x = self.avg_pool(x).flatten(1)
        x = self.dropout(x)
        x = self.fc(x)
        return x
def load_json_file(path: str) -> Any:
    with open(path, 'r') as f:
        data = json.load(f)
    return data
def dump_to_json_file(data: Any, path: str) -> None:
    """Dumping data to a json file.

    :param data: data for dumping
    :param path: path to the saved json file
    """
    with open(path, 'w') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
        f.write('\n')
def get_model():
    class2label = load_json_file(C2LP)
    label2class = {v: k for k, v in class2label.items()}
    model = SignsClassifier('efficientnet_b0', len(class2label))
    sdict = torch.load(MW)
    model.load_state_dict(sdict['state_dict'])
    model.eval()
    return model, label2class

def prep(img, img_size: Tuple[int, int] = (224, 224)) -> Callable:
    valid_transform = [
        albu.Resize(img_size[0], img_size[1]),
    ]
    img = albu.Compose(valid_transform)(image=img)['image']
    img = img.astype(np.float32)
    img /= 255
    img = np.transpose(img, (2, 0, 1))
    img -= np.array([0.485, 0.456, 0.406])[:, None, None]
    img /= np.array([0.229, 0.224, 0.225])[:, None, None]

    return img


def pred(IP, model, label2class):
    
    im = cv2.imread(IP)  # define your self. if cv2, after that use cv2.cvtColor(cv2.COLOR_BGR2RGB)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = prep(im, (224, 224))
    im = torch.from_numpy(im)
    im = im.unsqueeze(0)
    pred = model(im)
    pred = nn.LogSoftmax(dim=1)(pred)
    pred = pred.argmax(dim=1).numpy()[0]
    pred = label2class[pred
                       ]
    os.remove(IP)
    return pred

In [ ]:
model, l2c = get_model()

In [ ]:

 # path to image
dc_en = load_json_file(DCEN)
dc_ru = load_json_file(DCRU)
#mkdir('/tmp'); mkdir('/tmp/logs'); mkdir('/tmp/photos')

# Объект бота | bot object
bot = Bot(token="1902133157:AAH0tHhsqnvGw_O83rJFAy31MMHY7nC10VQ")
# Диспетчер для бота | bot dispatcher
dp = Dispatcher(bot)

@dp.message_handler(commands="start")
async def cmd_start(message: types.Message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    buttons = ["Русский", "English"]
    keyboard.add(*buttons)
    await message.answer('Здравствуйте, уважаемый. Это классификатор дорожных знаков для компании TwoGIS. Данный проект создавался в рамках состязания AIIJC. Пожалуйста, пришлите мне фото с дорожным знаком и ждите результата. Я работаю с 224х224 фотографиями, но вы можете прислать любой другой размер тоже. Для обратной связи вы можете написать на наш e-mail: cspdarknet53team@gmail.com\nIf you want to change language press the button "English"', reply_markup=keyboard)

@dp.message_handler(lambda message: message.text == "English")
async def with_puree(message: types.Message):
    user_lang[message.from_user.id] = 'en'
    
    await message.reply("Hello, dear customer. It is me, TwoGIS road sign classification system. This project was created for AIIJC competition. I am classifying road sign. Please, send me road sign image and wait for results. I am working with 224x224 image. But you can send me any other sized picture. You can send feedback to cspdarknet53team@gmail.com")
      
@dp.message_handler(lambda message: message.text == "Русский")
async def without_puree(message: types.Message):
    user_lang[message.from_user.id] = 'ru'
    await message.reply("Здравствуйте, уважаемый. Это классификатор дорожных знаков для компании TwoGIS. Данный проект создавался в рамках состязания AIIJC. Пожалуйста, пришлите мне фото с дорожным знаком и ждите результата. Я работаю с 224х224 фотографиями, но вы можете прислать любой другой размер тоже. Для обратной связи вы можете написать на наш e-mail: cspdarknet53team@gmail.com")
    
@dp.message_handler(content_types=["photo"])
async def download_photo(message: types.Message):
    
    await message.photo[-1].download(destination="./")
    res = pred('./photos/'+os.listdir('./photos')[0],
                                          model, 
                                          l2c)
    if user_lang[message.from_user.id] is 'en':
      await message.answer('Successfully uploaded.')
      await message.answer('Predicting...')
      await message.answer('This sign is ' + dc_en[res])
    else:
      await message.answer('Успешно загружено.')
      await message.answer('Обрабатывается...')
      await message.answer('Это знак ' + dc_ru[res])

@dp.message_handler(commands="feedback")
async def send_feedback(message: types.Message):
  if user_lang[message.from_user.id] == 'en':
    await message.answer(message.text)
      
    





In [ ]:
set_global_seed(42)

In [ ]:

if __name__ == "__main__":
    # Запуск бота
    executor.start_polling(dp, skip_updates=True)